# 프로필에서 키워드 추출하기

주어진 문서에서 키워드 추출하기

In [1]:
from typing import List

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

/Users/hyeonjinho/.pyenv/versions/3.11.6/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.8)

In [3]:
class Keywords(BaseModel):
    keyword_list: List[str] = Field(description="키워드 리스트")

In [4]:
parser = JsonOutputParser(pydantic_object=Keywords)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"keyword_list": {"title": "Keyword List", "description": "키워드 리스트", "type": "array", "items": {"type": "string"}}}, "required": ["keyword_list"]}\n```'


### partials의 사용법 변경

#### langchain==0.1.0 langchain-core==0.1.7

```python
human_prompt_template = HumanMessagePromptTemplate.from_template(
                            "{doc}\n{format_instructions}",
                            partial_variables={"format_instructions": format_instructions})

prompt = ChatPromptTemplate.from_messages(
    [
        human_prompt_template,
    ])
```

#### langchain=0.1.9 langchain-core==0.1.26

```python
human_prompt_template = HumanMessagePromptTemplate.from_template(
                            "{doc}\n{format_instructions}")

prompt = ChatPromptTemplate.from_messages(
    [
        human_prompt_template,
    ])
prompt = prompt.partial(format_instructions=format_instructions)
```

In [5]:
human_prompt_template = HumanMessagePromptTemplate.from_template(
                            "{doc}\n위 자기소개에서 키워드를 추출해줘\n{format_instructions}")

prompt = ChatPromptTemplate.from_messages(
    [
        human_prompt_template,
    ])
prompt = prompt.partial(format_instructions=format_instructions)

In [6]:
extract_keyword_chain = prompt | model | parser

In [7]:
profile = model.invoke("20대 평범한 직장인 자기소개 하나 만들어줘").content
print(profile)

안녕하세요! 저는 20대 평범한 직장인으로서 일상적인 업무를 열심히 수행하며 조직 내에서 소통과 협업을 중요시하는 직장인입니다. 전공과 관련된 분야에서 근무하고 있으며, 열정적이고 책임감 있는 태도로 업무에 임하고 있습니다. 항상 새로운 것을 배우고 성장하기 위해 노력하며, 동료들과 함께 일하는 것을 즐기고 있습니다. 현재의 직무에서 더 나은 결과를 이루기 위해 노력하며, 조직의 발전에 기여하기 위해 노력하고 있습니다. 함께 일하게 된다면 긍정적이고 협력적인 모습을 보여드리겠습니다. 감사합니다.


In [8]:
extract_keyword_chain.invoke({'doc': profile})

{'keyword_list': ['20대',
  '평범한 직장인',
  '일상적인 업무',
  '조직 내',
  '소통',
  '협업',
  '전공',
  '근무',
  '열정적',
  '책임감',
  '새로운 것',
  '배우기',
  '성장',
  '노력',
  '동료들',
  '즐기기',
  '직무',
  '더 나은 결과',
  '조직의 발전',
  '긍정적',
  '협력적']}